### Melon 100 chart 수집/분석/저장
* 100곡 노래의 제목,번호,노래상세정보url을 List에 저장하기 (정규표현식 사용)
* 노래의 상세정보를 추출해서 자료구조에 저장하고, json 파일로 저장하기
* json 파일을 Pandas의 read_json() 이용해서 DataFrame 객체로 저장하기
* DataFrame객체를 DB의 Table로 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re # Regular Expression
import pandas as pd
import json

In [19]:
url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

res = requests.get(url, headers=req_header)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("a[href*='playSong']")))
    print(len(soup.select("div#tb_list tr a[href*='playSong']")))
    a_tags = soup.select("a[href*='playSong']")
    
    # 100곡의 songs 정보를 저장할 List
    song_list = []
    for idx,a_tag in enumerate(a_tags,1):
        # 1곡의 song 정보를 저장할 dict
        song_dict = {}
        song_title = a_tag.text
        song_dict['song_title'] = song_title
        
        #print(idx, type(a_tag), a_tag, song_title)
        href_value = a_tag['href']
        #song id를 찾기 위한 정규표현식
        matched = re.search(r'(\d+)\);', href_value)
        if matched:
            song_id = matched.group(1)
            song_dict['song_id'] = song_id
                
            song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
            song_dict['song_url'] = song_detail_url
        song_list.append(song_dict)

print(len(song_list))
print(song_list[0:3])

200
100
100
100
[{'song_title': 'GANADARA (Feat. 아이유)', 'song_id': '34752700', 'song_url': 'https://www.melon.com/song/detail.htm?songId=34752700'}, {'song_title': 'INVU', 'song_id': '34626109', 'song_url': 'https://www.melon.com/song/detail.htm?songId=34626109'}, {'song_title': '사랑은 늘 도망가', 'song_id': '34061322', 'song_url': 'https://www.melon.com/song/detail.htm?songId=34061322'}]


#### 100개의 노래 상세정보 추출하기

In [46]:
import requests
from bs4 import BeautifulSoup
import re # Regular Expression
import pandas as pd
import json

req_header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}

song_detail_list = []
for idx,song in enumerate(song_list,1):
    song_detail_dict = {}
    
    song_detail_url = song['song_url']
    res = requests.get(song_detail_url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        #곡명
        song_detail_dict['곡명'] = song['song_title']
        
        print(idx , song['song_title'])
        
        #가수이름
        singer_span = soup.select("a[href*='goArtistDetail'] span")
        if singer_span:            
            song_detail_dict['가수'] = singer_span[0].text
        
        #앨범,발매일,장르
        song_dd = soup.select('div.meta dd')    
        if song_dd:
            song_detail_dict['앨범'] = song_dd[0].text
            song_detail_dict['발매일'] = song_dd[1].text
            song_detail_dict['장르'] = song_dd[2].text
        
        #좋아요 건수
        song_id = song['song_id']
        ajax_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'    
        res = requests.get(ajax_url, headers=req_header)
        if res.ok:
            song_detail_dict['좋아요'] = res.json()['contsLike'][0]['CONTSID']
        
        #가사
        lyric_div = soup.select('div#d_video_summary')
        if lyric_div:
            lyric_temp = lyric_div[0].text
            # \n\r\t 특수문자를 찾아주는 Pattern 객체생성
            pattern = re.compile(r'[\r\n\t]')
            # 특수문자를 ''(empty string) 으로 대체(substitute)해라
            lyric = pattern.sub('', lyric_temp.strip())                        
        else: #가사가 없는 경우
            lyric = ''
        
        song_detail_dict['가사'] = lyric
        
        #song_detail_dict를 song_detail_list에 추가
        song_detail_list.append(song_detail_dict)

print(len(song_detail_list))        
song_detail_list[0:2]        

1 GANADARA (Feat. 아이유)
2 INVU
3 사랑은 늘 도망가
4 듣고 싶을까
5 취중고백
6 사랑인가 봐
7 RUN2U
8 TOMBOY
9 abcdefu
10 언제나 사랑해
11 ELEVEN
12 이제 나만 믿어요
13 SMILEY (Feat. BIBI)
14 너, 너 (N번째 연애 X 휘인 (Whee In))
15 다정히 내 이름을 부르면
16 회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)
17 Step Back
18 신호등
19 다시 사랑한다면 (김필 Ver.)
20 어제처럼
21 그대라는 사치
22 STAY
23 별빛 같은 나의 사랑아
24 리무진 (Feat. MINO) (Prod. GRAY)
25 호랑수월가
26 눈이 오잖아(Feat.헤이즈)
27 HERO
28 잊었니
29 Next Level
30 끝사랑
31 Counting Stars (Feat. Beenzino)
32 strawberry moon
33 드라마
34 Dreams Come True
35 Bk Love
36 Weekend
37 Butter
38 Dynamite
39 존재만으로
40 너를 생각해
41 Permission to Dance
42 Savage
43 노래 (The Song)
44 바라만 본다
45 Can't Control Myself
46 내 생에 아름다운
47 겨울잠
48 만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL)
49 OHAYO MY NIGHT
50 라일락
51 잊을 수 있을까
52 서랍
53 계단말고 엘리베이터
54 My Universe
55 Christmas Tree
56 Celebrity
57 문득
58 우리가 헤어져야 했던 이유
59 찰나가 영원이 될 때 (The Eternal Moment)
60 언덕나무
61 그래서 그래 (Feat. 윤하)
62 사랑인걸
63 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
64 Thank You
65 봄날
66 너 아니면 

[{'곡명': 'GANADARA (Feat. 아이유)',
  '가수': '박재범',
  '앨범': 'GANADARA',
  '발매일': '2022.03.11',
  '장르': 'R&B/Soul',
  '좋아요': 34752700,
  '가사': '길거리를 걷다 보면사랑 노래만 흘러나와나는 왜 저런 게 낯설까난 한국말까지 서툴러번역기도 전혀 도움 안 돼네 맘엔 어떨까 걱정만어떤 단어를 쓸지 I don’t know어떤 말을 할지 I don’t know나는 왜 이런지 I don’t knowYou’re on my mind어떤 단어를 쓸지 I don’t know어떤 말을 할지 I don’t know나는 왜 이런지 I don’t knowYou’re on my mindBaby 정말 생각 안 나Body language로 자신 있어보여줄게girl just give me some your time너를 보면 손에 땀 나긴장 풀게 한 번 웃어줘Let me take u on a magic carpet ride손을 잡아 따라와맑은 날씨 보름달가르쳐줘 오늘 밤가나다라마바사손을 잡아 따라와맑은 날씨 보름달가르쳐줘 오늘 밤가나다라마바사너무 완벽하지 않아도걱정 마 네 맘을 조금 알 것 같아네게 좀 더 다가서 볼까고민해 매일 밤길거리를 걷다 보면계속 계속 네 생각이 나이런 내 모습이 왜 낯설까번역기 위에서헤매는 네 손가락까지자꾸만 생각이 나는 걸Body language도 나는 괜찮아But 생각 안 나면 그냥 웃어줘모든 것을 알려줄게 오늘 밤내가 나쁜 남자처럼 보여도너를 볼 때면 맘이 여려져I wouldn’t mind if I died in your arms손을 잡아 따라와맑은 날씨 보름달가르쳐줘 오늘 밤가나다라마바사손을 잡아 따라와맑은 날씨 보름달가르쳐줘 오늘 밤가나다라마바사손을 잡아 따라와맑은 날씨 보름달가르쳐줘 오늘 밤가나다라마바사손을 잡아 따라와맑은 날씨 보름달가르쳐줘 오늘 밤가나다라마바사'},
 {'곡명': 'INVU',
  '가수': '태연 (TAEYEON)',
  '앨범': 'INV

In [47]:
import json

with open('data/songs.json','w', encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [51]:
#song_detail_list 를 읽어서 DataFrame 객체 생성하는 방법
# [{},{},{}] {} => Series 객체, [] => DataFrame 객첸

import pandas as pd

song_list_df = pd.DataFrame(columns=['곡명','가수','앨범','발매일','장르','좋아요','가사'])

for song_detail in song_detail_list:
    series_obj = pd.Series(song_detail)
    song_list_df = song_list_df.append(series_obj, ignore_index=True)
    
song_list_df.head(3)    

,곡명,가수,앨범,발매일,장르,좋아요,가사
0,GANADARA (Feat. 아이유),박재범,GANADARA,2022.03.11,R&B/Soul,34752700,길거리를 걷다 보면사랑 노래만 흘러나와나는 왜 저런 게 낯설까난 한국말까지 서툴러번...
1,INVU,태연 (TAEYEON),INVU - The 3rd Album,2022.02.14,댄스,34626109,Falling in love너에겐 난 Option시작부터 다른 너와 나깨지는 Hea...
2,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",34061322,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...


#### json file을 DataFrame 객체로 저장하기

In [48]:
import pandas as pd

song_df = pd.read_json('data/songs.json')
print(type(song_df))
song_df.head()

<class 'pandas.core.frame.DataFrame'>


,곡명,가수,앨범,발매일,장르,좋아요,가사
0,GANADARA (Feat. 아이유),박재범,GANADARA,2022.03.11,R&B/Soul,34752700,길거리를 걷다 보면사랑 노래만 흘러나와나는 왜 저런 게 낯설까난 한국말까지 서툴러번...
1,INVU,태연 (TAEYEON),INVU - The 3rd Album,2022.02.14,댄스,34626109,Falling in love너에겐 난 Option시작부터 다른 너와 나깨지는 Hea...
2,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2,2021.10.11,"발라드, 국내드라마",34061322,눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...
3,듣고 싶을까,MSG워너비(M.O.M),듣고 싶을까,2022.02.26,발라드,34701627,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...
4,취중고백,김민석 (멜로망스),취중고백,2021.12.19,발라드,34431086,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...
